In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [2]:
#Vol indices
option_iv = pd.read_csv('options_iv.csv', index_col=0, parse_dates=True)
option_iv


,log_VXN,log_VIX
Date,,
2003-01-02,3.851211,3.234355
2003-01-03,3.822317,3.205993
2003-01-06,3.771611,3.215269
2003-01-07,3.760269,3.224062
2003-01-08,3.796837,3.239854
...,...,...
2023-12-22,2.810607,2.567254
2023-12-26,2.833213,2.564180
2023-12-27,2.786245,2.520113


In [3]:
apple_data = pd.read_csv('/Users/raphaelravinet/Code/BSE/Thesis/apple/AAPL_realized_combined.csv', index_col=0, parse_dates=True)
apple_data.drop(columns = ['daily_realised_log_var_realized_var'],inplace=True)
apple_data.rename(columns = {'daily_realised_log_var_realized_var_filtered':'daily_realised_log_var'}, inplace=True)

In [4]:
apple_data

,daily_realised_log_var
Day,
2003-09-10,0.000439
2003-09-11,0.000512
2003-09-12,0.000319
2003-09-15,0.000638
2003-09-16,0.000497
...,...
2024-01-25,0.000127
2024-01-26,0.000052
2024-01-29,0.000095


In [5]:
apple_data['Daily_Vol'] = apple_data['daily_realised_log_var'].shift(1)
apple_data['Weekly_Vol'] = apple_data['daily_realised_log_var'].rolling(window=5).mean().shift(1)
apple_data['Monthly_Vol'] = apple_data['daily_realised_log_var'].rolling(window=22).mean().shift(1)
apple_data.dropna(inplace=True)
apple_data

,daily_realised_log_var,Daily_Vol,Weekly_Vol,Monthly_Vol
Day,,,,
2003-10-10,0.000433,0.001190,0.000784,0.000717
2003-10-13,0.000281,0.000433,0.000585,0.000717
2003-10-14,0.000245,0.000281,0.000582,0.000706
2003-10-15,0.000534,0.000245,0.000497,0.000703
2003-10-16,0.003877,0.000534,0.000537,0.000698
...,...,...,...,...
2024-01-25,0.000127,0.000083,0.000225,0.000187
2024-01-26,0.000052,0.000127,0.000123,0.000184
2024-01-29,0.000095,0.000052,0.000120,0.000185


In [6]:
apple_data_with_IV = apple_data.merge(option_iv[['log_VXN', 'log_VIX']], left_index=True, right_index=True)
apple_data_with_IV


,daily_realised_log_var,Daily_Vol,Weekly_Vol,Monthly_Vol,log_VXN,log_VIX
2003-10-10,0.000433,0.001190,0.000784,0.000717,3.318540,2.915064
2003-10-13,0.000281,0.000433,0.000585,0.000717,3.287282,2.865054
2003-10-14,0.000245,0.000281,0.000582,0.000706,3.276767,2.854745
2003-10-15,0.000534,0.000245,0.000497,0.000703,3.287282,2.873000
2003-10-16,0.003877,0.000534,0.000537,0.000698,3.237501,2.844328
...,...,...,...,...,...,...
2023-12-22,0.000030,0.000192,0.000092,0.000109,2.810607,2.567254
2023-12-26,0.000025,0.000030,0.000080,0.000108,2.833213,2.564180
2023-12-27,0.000050,0.000025,0.000070,0.000101,2.786245,2.520113
2023-12-28,0.000065,0.000050,0.000069,0.000101,2.794839,2.523326


In [7]:
#Standard HAR
X = apple_data.iloc[:,1:]
X = sm.add_constant(X)
y = apple_data.iloc[:,0]
T = len(X)
nw_lag = int(4 * (T / 100) ** (2 / 9))
HAR_model = sm.OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags' :nw_lag,'use_correction':True})
HAR_result = HAR_model.summary()

In [8]:
HAR_result

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     daily_realised_log_var   R-squared:                       0.213
Model:                                OLS   Adj. R-squared:                  0.213
Method:                     Least Squares   F-statistic:                     44.19
Date:                    Thu, 18 Apr 2024   Prob (F-statistic):           3.48e-28
Time:                            14:06:25   Log-Likelihood:                 28887.
No. Observations:                    5111   AIC:                        -5.777e+04
Df Residuals:                        5107   BIC:                        -5.774e+04
Df Model:                               3                                         
Covariance Type:                      HAC                                         
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const        9.924e-05   2.55e-05      3.898      0.000    4.93e-05       0.000
Daily_Vol       0.1437      0.038      3.795      0.000       0.069       0.218
Weekly_Vol      0.3584      0.088      4.072      0.000       0.186       0.531
Monthly_Vol     0.2652      0.071      3.756      0.000       0.127       0.404
==============================================================================
Omnibus:                     7996.722   Durbin-Watson:                   2.014
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          4785172.923
Skew:                           9.923   Prob(JB):                         0.00
Kurtosis:                     151.580   Cond. No.                     3.64e+03
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 9 lags and with small sample correction
[2] The condition number is large, 3.64e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [13]:
apple_data_with_IV.iloc[:,1:5]

,Daily_Vol,Weekly_Vol,Monthly_Vol,log_VXN
2003-10-10,0.001190,0.000784,0.000717,3.318540
2003-10-13,0.000433,0.000585,0.000717,3.287282
2003-10-14,0.000281,0.000582,0.000706,3.276767
2003-10-15,0.000245,0.000497,0.000703,3.287282
2003-10-16,0.000534,0.000537,0.000698,3.237501
...,...,...,...,...
2023-12-22,0.000192,0.000092,0.000109,2.810607
2023-12-26,0.000030,0.000080,0.000108,2.833213
2023-12-27,0.000025,0.000070,0.000101,2.786245
2023-12-28,0.000050,0.000069,0.000101,2.794839


In [12]:
#HAR-IV
X_IV = apple_data_with_IV.iloc[:,1:5]
X_IV = sm.add_constant(X_IV)
y_IV = apple_data_with_IV.iloc[:,0]
T = len(y_IV)
nw_lag = int(4 * (T / 100) ** (2 / 9))
HAR_IV_model = sm.OLS(y_IV, X_IV).fit(cov_type='HAC', cov_kwds={'maxlags' :nw_lag,'use_correction':True})
HAR_IV_results = HAR_IV_model.summary()
HAR_IV_results

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     daily_realised_log_var   R-squared:                       0.224
Model:                                OLS   Adj. R-squared:                  0.223
Method:                     Least Squares   F-statistic:                     35.27
Date:                    Thu, 18 Apr 2024   Prob (F-statistic):           4.21e-29
Time:                            14:24:34   Log-Likelihood:                 28795.
No. Observations:                    5090   AIC:                        -5.758e+04
Df Residuals:                        5085   BIC:                        -5.755e+04
Df Model:                               4                                         
Covariance Type:                      HAC                                         
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -0.0010      0.000     -4.928      0.000      -0.001      -0.001
Daily_Vol       0.1384      0.037      3.790      0.000       0.067       0.210
Weekly_Vol      0.3438      0.087      3.959      0.000       0.174       0.514
Monthly_Vol     0.1490      0.072      2.081      0.037       0.009       0.289
log_VXN         0.0004   7.11e-05      5.391      0.000       0.000       0.001
==============================================================================
Omnibus:                     7999.201   Durbin-Watson:                   2.006
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          4841314.058
Skew:                          10.012   Prob(JB):                         0.00
Kurtosis:                     152.755   Cond. No.                     1.20e+04
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 9 lags and with small sample correction
[2] The condition number is large, 1.2e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
apple_data['Predicted_RV'] = HAR_model.predict(X)

plt.figure(figsize=(14, 8))
plt.plot(apple_data.index, apple_data.iloc[:, 0], 'r-', label='Observed RV', alpha=0.5)
plt.plot(apple_data.index, apple_data['Predicted_RV'], 'b-', label='Forecasted RV', alpha=0.7)
plt.title('Observed and Forecasted RV based on HAR Model')
plt.legend()
plt.show()

In [ ]:
# class Model_Preparation:
    
#     def  __init__ (self, data):
#         self.data = data
        
#     def prepare_data(self):
#         self.data['Daily_Vol'] = self.data['daily_realised_log_var'].shift(1)
#         self.data['Weekly_Vol'] = self.data['daily_realised_log_var'].rolling(window=5).mean().shift(1)
#         self.data['Monthly_Vol'] = self.data['daily_realised_log_var'].rolling(window=22).mean().shift(1)
#         self.data.dropna(inplace=True)
#         return self.data

In [ ]:
# class HAR_estimation:
#     """ this takes as argument a dataframe with the following columns:
#     Daily_vol: daily realised volatility
#     Weekly_vol: weekly realised volatility
#     Monthly_vol: monthly realised volatility
#     """
    
#     def __init__(self, data, implied_vol = False, jc_component = False):
#         self.data = data
#         self.implied_vol = implied_vol
#         self.jc_component = jc_component
    
        
    
#     def processing_data(self):
    
#     data.drop(columns = ['daily_realised_var'],inplace=True)
#     data['Daily_Vol'] = data['daily_realised_log_var'].shift(1)
#     data['Weekly_Vol'] = data['daily_realised_log_var'].rolling(window=5).mean().shift(1)
#     data['Monthly_Vol'] = data['daily_realised_log_var'].rolling(window=22).mean().shift(1)
#     data = data.dropna()
    
#     if self.implied_vol == True:
#         data = data.merge(option_iv[['log_VXN', 'log_VIX']], left_index=True, right_index=True)
#         X = data.iloc[:, 1:]
#         y = data['daily_realised_log_var']
#         T = len(X)
        
        
#     def har_estiation(self):
#     X = sm.add_constant(X)
#     max_lag = int(T ** (1 / 4))
#     model = sm.OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags': max_lag})
#     results = model.summary()
        
            
#     return results
    